In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the data

In [2]:
path_to_file = tf.keras.utils.get_file('alice.txt', 'https://raw.githubusercontent.com/donaldong/rnn-text-gen/master/data/alice.txt')

163840/147526 [=================================] - 0s 0us/step


Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 147524 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

﻿ALICE'S ADVENTURES IN WONDERLAND

Lewis Carroll

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister 


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

71 unique characters


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [7]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
A
L
I
C


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"\ufeffALICE'S ADVENTURES IN WONDERLAND\r\n\r\nLewis Carroll\r\n\r\nCHAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was be"
'ginning to get very tired of sitting by her sister on the\r\nbank, and of having nothing to do: once or'
' twice she had peeped into the\r\nbook her sister was reading, but it had no pictures or conversations '
"in\r\nit, 'and what is the use of a book,' thought Alice 'without pictures or\r\nconversations?'\r\n\r\nSo sh"
'e was considering in her own mind (as well as she could, for the\r\nhot day made her feel very sleepy a'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [9]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "\ufeffALICE'S ADVENTURES IN WONDERLAND\r\n\r\nLewis Carroll\r\n\r\nCHAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was b"
Target data: "ALICE'S ADVENTURES IN WONDERLAND\r\n\r\nLewis Carroll\r\n\r\nCHAPTER I. Down the Rabbit-Hole\r\n\r\nAlice was be"


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [11]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [53]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [54]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [55]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 71) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [56]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           18176     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_7 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dropout_6 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense_6 (Dense)              (64, None, 71)            72775     
Total params: 10,326,855
Trainable params: 10,326,855
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [57]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([ 4, 19, 60,  8, 21, 42, 62,  5, 68, 40, 32, 32, 13, 10, 64, 37,  4,
       33, 48, 33, 15, 27, 12, 33, 55, 42, 24, 36, 63, 42, 38, 24, 25, 22,
       30,  9, 61, 43, 45, 58, 58, 16,  8,  4, 60, 32, 35, 29, 67, 65, 45,
       29,  8, 20, 15, 18, 53, 47, 53,  4,  6, 33, 41, 21, 20, 70,  5, 52,
        6, 41, 44, 49, 33, 61, 46,  7,  7, 69, 51, 65, 14,  5, 61, 52, 41,
       68, 19, 33, 17, 67,  8,  5, 18, 70, 63, 61, 25, 16, 58,  4])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [58]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 71)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.2627764


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [59]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [60]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [61]:
EPOCHS=20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
22/22 [==============================] - 10s 308ms/step - loss: 3.7882
Epoch 2/20
22/22 [==============================] - 7s 309ms/step - loss: 3.0905
Epoch 3/20
22/22 [==============================] - 7s 309ms/step - loss: 2.7040
Epoch 4/20
22/22 [==============================] - 7s 309ms/step - loss: 2.4132
Epoch 5/20
22/22 [==============================] - 7s 307ms/step - loss: 2.2437
Epoch 6/20
22/22 [==============================] - 7s 307ms/step - loss: 2.0980
Epoch 7/20
22/22 [==============================] - 7s 307ms/step - loss: 1.9763
Epoch 8/20
22/22 [==============================] - 7s 305ms/step - loss: 1.8747
Epoch 9/20
22/22 [==============================] - 7s 304ms/step - loss: 1.7825
Epoch 10/20
22/22 [==============================] - 7s 305ms/step - loss: 1.6974
Epoch 11/20
22/22 [==============================] - 7s 305ms/step - loss: 1.6198
Epoch 12/20
22/22 [==============================] - 7s 304ms/step - loss: 1.5482
Epoch 13/20
22/22 [=====

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [62]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            18176     
_________________________________________________________________
gru_8 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_9 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dropout_7 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_7 (Dense)              (1, None, 71)             72775     
Total params: 10,326,855
Trainable params: 10,326,855
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [67]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [69]:
print(generate_text(model, start_string=u"ALICE'S "))

ALICE'S CHAPTER VII. A March Hare.

'I don't were all the best what a little sigher, and he had gone and herself up and was the only one of the same as
she spoke.

'I don't know behind it,' said the Cat, and she said to herself, 'I wonder what they had a great hurry; 'and then they'll be a ran off such a thing to herself, 'I don't talk about me in that say! I wonder a
for a mouse, that you like a strange to the Gryphon. 'I suppose the carried
about it, who were ine on the baby stupid?'

'I should like the Gryphon as it didn't such a treecul to speak to the Mock Turtle said wonder in the air. I'm afraid,
    Beautiful Soup!' said the Mock Turtle.

'I'll tell me the moral of the spy, he see it fartle and walked off
and dropped, and the poor little thing as the roses
all the trues tarts easing in the party was a good deal of the baby, the three gardeners was
to ask it, and the hall was gone and the baby with his head!' and then at
she spoke. 'I never have no ndistence, and was per
